# LangChain LCEL

## 사전 준비 가상 환경 구축, 패키지 설치 등

In [13]:
#1

!pip uninstall -y google-generativeai
!pip install langchain==0.3.20 langchain_core==0.3.49 langchain_google_genai==2.1.1 httpx==0.28.1 requests==2.32.3

## Chain 연결

In [15]:
#2

import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI

#환경 변수 및 패키지 준비
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
#os.environ["GOOGLE_API_KEY"] = "***"


In [16]:
#3

# prompt 템플릿 정의
message1 = [
        SystemMessage(content= "당신은 한국어를 말하는 우수한 어시스턴트입니다. 회답에는 반드시 한국어로 대답해 주세요. 또 생각하는 과정도 출력해 주세요."),
        HumanMessagePromptTemplate.from_template(
            [
                {
                    "type": "text",
                    "text": "{question}"
                },
            ]
        )
    ]

prompt1 = ChatPromptTemplate.from_messages(message1)

# gemini 모델 정의
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
)

In [17]:
#4

#출력 parser 정의
from langchain_core.output_parsers import StrOutputParser

# 체인 정의
chain1 = prompt1 | model | StrOutputParser()

In [18]:
#5

# 사용자로부터 입력
user_input = "일에 대한 열정을 되찾기 위한 5가지 아이디어를 제시해 주세요."

# 체인 실행
output = chain1.invoke({"question": user_input})
print("사용자 입력:", user_input)
print("결과:", output)

사용자 입력: 일에 대한 열정을 되찾기 위한 5가지 아이디어를 제시해 주세요.
결과: 알겠습니다. 일에 대한 열정을 되찾기 위한 5가지 아이디어를 제시해 드리겠습니다. 먼저, 열정을 잃은 원인을 파악하고, 그에 맞는 해결책을 찾는 것이 중요합니다. 다음은 제가 생각하는 5가지 아이디어입니다.

**1. 새로운 목표 설정 및 작은 성공 경험:**

*   **생각 과정:** 현재 업무에 매몰되어 있다면, 장기적인 목표를 잊기 쉽습니다. 새로운 목표를 설정함으로써 다시 동기 부여를 얻을 수 있습니다. 또한, 큰 목표보다는 작은 목표를 설정하고 달성해 나가는 과정에서 성취감을 느끼는 것이 중요합니다.
*   **구체적인 방법:**
    *   자신의 커리어 목표를 재검토하고, 현재 업무와 연결될 수 있는 새로운 목표를 설정합니다.
    *   목표를 세분화하여 작은 단계로 나누고, 각 단계를 완료할 때마다 자신에게 보상을 제공합니다.
    *   업무 관련 새로운 기술을 배우거나 자격증을 취득하는 것을 목표로 설정할 수도 있습니다.

**2. 업무 환경 변화 시도:**

*   **생각 과정:** 단조로운 업무 환경은 지루함을 유발하고 열정을 잃게 만들 수 있습니다. 환경 변화는 새로운 자극을 제공하고, 업무에 대한 신선한 시각을 갖게 해줍니다.
*   **구체적인 방법:**
    *   사무실 내 자리 배치를 바꾸거나, 재택근무를 활용하여 업무 공간에 변화를 줍니다.
    *   업무 시간 중 짧은 휴식 시간을 활용하여 산책을 하거나, 새로운 장소에서 커피를 마시는 등 기분 전환을 시도합니다.
    *   업무 관련 컨퍼런스나 세미나에 참석하여 새로운 아이디어를 얻고, 다른 사람들과 교류합니다.

**3. 새로운 프로젝트 참여 또는 업무 범위 확장:**

*   **생각 과정:** 기존 업무에 익숙해져 있다면, 새로운 프로젝트에 참여하거나 업무 범위를 확장함으로써 새로운 도전 과제를 찾고, 자신의 역량을 개발할 수 있습니다.
*   **구체적인 방법:*

## 복수의 Chain을 연결

In [22]:
#6

user_prompt_template = """
문제, 정답, 채점 기준, 언어 모델이 생성한 답변이 주어집니다.

# 지시
'채점 기준'과 '정답 예'를 참고해, 응답을 1,2,3,4,5의 5단계로 채점합니다. 단, 먼저 한번, 스텝 바이 스텝으로 사고를 하고 나서 결론으로서 점수를 숫자로 채점해 주세요.
사고 과정도 출력하십시오.

# 문제
{question}

# 정답 예
{correct_text}

# 채점 기준
기본 채점 기준
- 1점: 잘못됨, 지시에 따르지 않음
- 2점: 잘못되어 있지만 방향성은 맞다
- 3점: 부분적으로 잘못됨, 부분적으로 적합함
- 4점: 맞다
- 5점: 유용하다

기본 감점 항목
- 부자연스러운 한국어: -1점
- 부분적으로 사실과 다른 내용을 말하고 있다: -1점
- '윤리적으로 대답할 수 없습니다'와 같이 과도하게 안전성을 신경 쓰고 있다 : 2점으로 한다

문제별 채점 기준
{eval_aspect}

# 언어 모델의 응답
{llm_output}

# 여기까지가 '언어 모델의 응답'입니다.

# 지시
'채점 기준'과 '정답 예'를 참고로 해, 응답을 1, 2, 3, 4, 5의 5단계로 채점합니다.
사고 과정도 출력해 주세요.

"""


# 질문의 정답 예
correct_text = """
1. 자신의 직업에 대한 관심을 재발견하기 위해 새로운 기술과 지식을 배우는 것.
2. 대학이나 세미나 등에서 강연을 듣는 것으로, 일에 대한 새로운 아이디어나 동기를 얻는 것.
3. 일에 스트레스를 느끼는 경우 스트레스 관리 기술을 배우세요.
4. 일 이외의 즐거운 활동을 함으로써 스트레스를 발산하는 것.
5. 일에 대해 자기 평가를 함으로써, 자신이 어떻게 진화하고 있는지를 아는 것.
"""

# 질문의 평가 기준
eval_aspect = """
- 열의를 되찾는 것이 아니라, 일의 효율화/스킬업과 같은 문맥으로 되어 있으면 1점 감점
- 제시한 아이디어가 5개보다 많아야 하고 적으면 1점 감점
- 5개의 아이디어 중 내용이 중복되어 있는 것이 있으면 1점 감점
"""


In [23]:
#7

# prompt 템플릿 정의
message2 = [
        HumanMessagePromptTemplate.from_template(
            [
                {
                    "type": "text",
                    "text": user_prompt_template
                },
            ]
        )
    ]

prompt2 = ChatPromptTemplate.from_messages(message2)

# 체인 정의
chain2 = prompt2 | model | StrOutputParser()

In [24]:
#8

from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

# chain 연결
chain = (
    RunnableParallel(
        {
            "question": RunnableLambda(lambda x: x["question"]) | RunnablePassthrough(),
            "correct_text": RunnableLambda(lambda x: x["correct_text"]) | RunnablePassthrough(),
            "eval_aspect": RunnableLambda(lambda x: x["eval_aspect"]) | RunnablePassthrough(),
            "llm_output":RunnableLambda(lambda x: x["question"]) | chain1,
        }
    ).assign(evaluation=chain2)
)

response = chain.invoke({"question": user_input, "correct_text": correct_text, "eval_aspect": eval_aspect})

print("########## 재가공 후 출력 ##########")
print("사용자 입력:", user_input)
print("첫번째 결과:", response["llm_output"])
print("\n########## 평가 ##########")
print(response["evaluation"])

print("\n########## 원시 출력 ##########")
print(response)


########## 재가공 후 출력 ##########
사용자 입력: 일에 대한 열정을 되찾기 위한 5가지 아이디어를 제시해 주세요.
첫번째 결과: 알겠습니다. 일에 대한 열정을 되찾기 위한 5가지 아이디어를 제시해 드리겠습니다. 먼저, 열정을 잃은 원인을 파악하고, 그에 맞는 해결책을 찾는 것이 중요합니다. 다음은 제가 생각하는 5가지 아이디어입니다.

**1. 새로운 목표 설정 및 작은 성공 경험:**

*   **생각 과정:** 현재 업무에 매몰되어 있다면, 장기적인 목표를 잊기 쉽습니다. 새로운 목표를 설정함으로써 다시 동기 부여를 얻을 수 있습니다. 또한, 큰 목표보다는 작은 목표를 설정하고 달성해 나가는 과정에서 성취감을 느끼는 것이 중요합니다.
*   **구체적인 방법:**
    *   자신의 커리어 목표를 재검토하고, 현재 업무와 연결될 수 있는 새로운 목표를 설정합니다.
    *   목표를 세분화하여 작은 단계로 나누고, 각 단계를 완료할 때마다 자신에게 보상을 제공합니다.
    *   업무 관련 새로운 기술을 배우거나 자격증을 취득하는 것을 목표로 설정할 수도 있습니다.

**2. 업무 환경 변화 시도:**

*   **생각 과정:** 단조로운 업무 환경은 지루함을 유발하고 열정을 잃게 만들 수 있습니다. 환경 변화는 새로운 자극을 제공하고, 업무에 대한 신선한 시각을 갖게 해줍니다.
*   **구체적인 방법:**
    *   사무실 내 자리 배치를 바꾸거나, 재택근무를 활용하여 업무 공간에 변화를 줍니다.
    *   업무 시간 중 짧은 휴식 시간을 활용하여 산책을 하거나, 새로운 장소에서 커피를 마시는 등 기분 전환을 시도합니다.
    *   업무 관련 컨퍼런스나 세미나에 참석하여 새로운 아이디어를 얻고, 다른 사람들과 교류합니다.

**3. 새로운 프로젝트 참여 또는 업무 범위 확장:**

*   **생각 과정:** 기존 업무에 익숙해져 있다면, 새로운 프로젝트에 참여하거나 업무 범위를 확장함으로써 새로운 도전 과제를 찾고, 